In [1]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from ipywidgets import interact

direc = r'D:\Documents\AHC-BME\PhD\mPAP - vortex\Results' #directory for patient-specific results

PS = pd.read_csv(direc+'\P-S_results.csv')
x_data = np.array(PS['mPAP'])
y_Lit = np.array(PS['t_vortex (lit)'])
y_CFD = np.array(PS['t_vortex (CFD)'])
y_4D = np.array(PS['t_vortex (4D)'])
y_data = [y_Lit, y_CFD, y_4D]

In [3]:
def interactive_selection_plot(x_data, y_data_sets, methods):
  """
  Creates an interactive plot with multiple vortex duration sets, allowing users to select which method to display.

  Args:
      x_data: Pressure values (mPAP, in mmHg)
      y_data_sets: A list of lists or arrays, each representing a method of vortex duration quantification.
  """

  def update_plot(**kwargs):
    """
    Updates the plot with the selected y-data sets and regression lines.

    Args:
      kwargs: A dictionary of checkbox names (e.g., 'checkbox_1') and their corresponding boolean values.
    """
    
    plt.figure(figsize=(8, 6))
    for i, (checkbox_name, selected) in enumerate(kwargs.items()):
      if selected:
        label = methods[i]
        plt.plot(x_data, y_data_sets[i], 'o', label=label)
        
        # Add regression line
        x_data_PH = x_data[10:]-16
        y_data_PH = y_data_sets[i][10:]
        y_data_true = y_data_sets[0][10:]
        reg = LinearRegression(fit_intercept=False).fit(x_data_PH.reshape(-1, 1), y_data_PH.reshape(-1, 1))
        score = reg.coef_*np.transpose(x_data_PH)
        r_squared = r2_score(y_data_true.reshape(-1, 1), y_data_PH.reshape(-1, 1))
        plt.plot(x_data_PH+16, np.transpose(score), '--', label=f'{label} Regression, R2 = %.2f'%r_squared)
        
        
    plt.xlabel('mPAP (mmHg)')
    plt.ylabel('Vortex Duration')
    plt.title('Interactive Selection Plot with Regression')
    plt.legend()
    plt.grid(True)
    plt.show()

  # Create checkboxes for each data set
  checkboxes = {}
  for i, y_data_set in enumerate(y_data_sets):
    checkbox = widgets.Checkbox(value=True, description=str(methods[i]))
    checkboxes[f'checkbox_{i+1}'] = checkbox

  # Create the interactive plot
  interact(update_plot, **checkboxes)


methods = ['Literature', 'CFD', '4D']
interactive_selection_plot(x_data, y_data, methods)

interactive(children=(Checkbox(value=True, description='Literature'), Checkbox(value=True, description='CFD'),…

In [4]:
def interactive_correspondence_plot(x_data, y_data_sets, methods):
  """
  Creates an interactive plot with correspondence between literature predicted vortex duration and either CFD or 4D methods,
  allowing users to select which method to display.

  Args:
      x_data: Literature predicted vortex duration
      y_data_sets: A list of lists or arrays, each representing a method of vortex duration quantification.
  """

  def update_plot(**kwargs):
    """
    Updates the plot with the selected y-data sets and regression lines.

    Args:
      kwargs: A dictionary of checkbox names (e.g., 'checkbox_1') and their corresponding boolean values.
    """
    
    plt.figure(figsize=(8, 6))
    for i, (checkbox_name, selected) in enumerate(kwargs.items()):
      if selected:
        label = methods[i]
        plt.plot(x_data, y_data_sets[i], 'o', label=label)
        
        # Add regression line
        reg = LinearRegression().fit(x_data.reshape(-1, 1), y_data_sets[i].reshape(-1,1))
        score = reg.coef_*np.transpose(x_data)
        r_squared = r2_score(x_data.reshape(-1, 1), y_data_sets[i].reshape(-1, 1))
        plt.plot(x_data, np.transpose(score), '--', label=f'{label} Regression, R2 = %.2f'%r_squared)
        
    plt.xlabel('Literature vortex duration')
    plt.ylabel('Method vortex duration')
    plt.title('Interactive Selection Plot with Regression')
    plt.legend()
    plt.grid(True)
    plt.show()

  # Create checkboxes for each data set
  checkboxes = {}
  for i, y_data_set in enumerate(y_data_sets):
    checkbox = widgets.Checkbox(value=True, description=str(methods[i]))
    checkboxes[f'checkbox_{i+1}'] = checkbox

  # Create the interactive plot
  interact(update_plot, **checkboxes)

new_y_data = [y_CFD, y_4D]
methods = ['CFD', '4D']
interactive_correspondence_plot(y_Lit, new_y_data, methods)

interactive(children=(Checkbox(value=True, description='CFD'), Checkbox(value=True, description='4D'), Output(…